# State altering agents: stand up comedy

In this example, we are going to create two agents. We'll provide them with `system prompts` to ask them to play the role of a comedian and we'll initiate a conversation between them. This time, this conversation will be a chat, instead of a simple `generate_reply()` request, meaning that each answer will alter the state of each agent. This example is a standard example that is used in the autogen documentation.

Let's get started!

In [ ]:
from autogen import ConversableAgent

from openai import OpenAI

import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Get the API key
api_key = os.getenv("OPENAI_API_KEY")

In [1]:
# We'll always have to start by creating a llm_config object to configure our agents
llm_config = {
    "model": "gpt-3.5-turbo", 
    "api_key": "api_key",
    "cache": None
    }

## Conversable Agent

Let's important the ConversableAgent class:

In [2]:
from autogen import ConversableAgent

Let us now define our agents, we'll give them names, our chatGPT3.5 config and a `system prompt` to let them know that they are a stand-up comedian who is part of a two-person comedy show.

In [3]:
bret = ConversableAgent(
    name="Bret",
    llm_config=llm_config,
    system_message="Your name is Bret and you are a stand-up comedian in a two-person comedy show.",
)
jemaine = ConversableAgent(
    name="Jemaine",
    llm_config=llm_config,
    system_message="Your name is Jemain and you are a stand-up comedian in a two-person comedy show.",
)

And now that we have our agents, we can start a chat between them. This time, we'll use the `initiate_chat()` function from one of the agents instead of the `generate_reply()`. This function will require a receiver and an initiation message. We will also specify a number of turns after what the conversation will stop.  
We will also store the result of this exchange in an object called `chat_result`.

In [4]:
chat_result = bret.initiate_chat(
    recipient = jemaine,
    message="Jemaine, tell me a joke.", 
    max_turns=2 # The conversation will stop after each agent has spoken twice
)

Bret (to Jemaine):

Jemaine, tell me a joke.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jemaine (to Bret):

Sure! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Bret (to Jemaine):

Nice one, Jemaine! Now, I've got a joke for you. Why did the math book look sad? Because it had too many problems!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jemaine (to Bret):

Haha, that's a good one! I guess that book could use some cheering up!

--------------------------------------------------------------------------------


You should have been able to read an exchange between the two agents! You will notice that they might for example remember each other's name and other elements about each other, their state is affected by the conversation. This is your first conversation between LLM agents, congrats!  
You can have some fun if you'd like by creating two different agents, give them different roles, historical ones, different setups and let them talk to practice.  

## Better explore chat results

During this exchange, the only element we received is the chat exchange. We might want to further explore it. We can do so using the following elements.

We are going to import the `pprint` standard library from python to display somee elements of the chat exchange. We can start by displaying the chat history:

In [5]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': 'Jemaine, tell me a joke.', 'role': 'assistant'},
 {'content': 'Sure! Why did the scarecrow win an award? Because he was '
             'outstanding in his field!',
  'role': 'user'},
 {'content': "Nice one, Jemaine! Now, I've got a joke for you. Why did the "
             'math book look sad? Because it had too many problems!',
  'role': 'assistant'},
 {'content': "Haha, that's a good one! I guess that book could use some "
             'cheering up!',
  'role': 'user'}]


This gives us the whole exchange in a structured format that can be exported or re-used elsewhere in our program.

We can also get a summary of how much this chat has cost us:

In [6]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 70,
                                                             'cost': 0.00020400000000000003,
                                                             'prompt_tokens': 198,
                                                             'total_tokens': 268},
                                      'total_cost': 0.00020400000000000003},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 70,
                                                             'cost': 0.00020400000000000003,
                                                             'prompt_tokens': 198,
                                                             'total_tokens': 268},
                                      'total_cost': 0.00020400000000000003}}


And finally we can also have a summary of the chat, but in this case, we are using the default mode which means that the summary is the last message... 

In [7]:
pprint.pprint(chat_result.summary)

"Haha, that's a good one! I guess that book could use some cheering up!"


Since it would be better to have a summary that is a real summary and not the last message, we're going to re-run this exchange by specifying an additional argument to have a real summary of the exchange.

## Chat summary

We are going to re-run the initiation of the chat, without re-defining the agents, but this time will add two arguments:

In [8]:
chat_result = bret.initiate_chat(
    recipient = jemaine, 
    message="I'm Bret. Jemaine, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm", # Can be "last_message" (DEFAULT) or "reflection_with_llm"
    summary_prompt="Summarize the conversation", # We specify the prompt used to summarize the chat
)

Bret (to Jemaine):

I'm Bret. Jemaine, let's keep the jokes rolling.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jemaine (to Bret):

Thanks, Bret! So, I was at the doctor's office the other day and he told me I needed to start exercising more. I was like, "Doc, I walk to the fridge all the time, doesn't that count?"

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Bret (to Jemaine):

Haha, walking to the fridge definitely counts as exercise in my book! But I think the doctor might have had something a little more strenuous in mind. Maybe try a brisk walk to the grocery store next time!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jemaine (to Bret):

Yeah, maybe I'll try that next time, Bret. But hey, at least I'm getting my steps in, right? Who needs a Fitbit when you have a fridge that's co

And this time, if we take a look at the summary, the result we'll get is the same as if the whole conversation was sent to chatGPT3.5 with the prompt "Summarize the conversation":

In [9]:
pprint.pprint(chat_result.summary)

('Bret and Jemaine joke about getting exercise by walking to the fridge and '
 'suggest trying a brisk walk to the grocery store.')


## Termination

Finally, the last element we'll explore about a two-person chat is how to end the conversation. Until now, we've used the argument `max_turns=2` to end the conversation after two turns. But we could also let the agents decide when they're done and finish the conversation then. 

To do that, we will have to tell each agent which words they should use when they're done and we'll have to monitor their messages for those words. Once autogen detects that the agent sent those words, the conversation will end. 

Since this is an agent setting, we'll have to re-define our agents this time:

In [10]:
bret = ConversableAgent(
    name="Bret",
    llm_config=llm_config,
    system_message="Your name is Bret and you are a stand-up comedian in a two-person comedy show. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)
jemaine = ConversableAgent(
    name="Jemaine",
    llm_config=llm_config,
    system_message="Your name is Jemain and you are a stand-up comedian in a two-person comedy show. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

Note how we told each agent to tell us when they're done with "*When you're ready to end the conversation, say 'I gotta go'.*" and how we added an `is_termination_msg` argument that looks into the sequence of characters `I gotta go` in each message using a python `lambda` function.

Ok, let's now run this chat, and this time we will not specify a `max_turns=2`.  
We will also not specify a LLM based summary method because we won't use it and running it will cost us some token, so we'll only specify it if we need it.

In [12]:
chat_result = bret.initiate_chat(
    recipient = jemaine, 
    message="I'm Bret. Jemaine, let's keep the jokes rolling, let's start with jokes about therapists.", 
)

Bret (to Jemaine):

I'm Bret. Jemaine, let's keep the jokes rolling, let's start with jokes about therapists.

--------------------------------------------------------------------------------
Jemaine (to Bret):

Hey, Bret! Sure thing, let's dive into the world of therapists... You know, therapists always ask you, "How does that make you feel?" So I'm like, "Well, probably the same way it made the last person feel, but with a different accent." It's like they have a template or something.

--------------------------------------------------------------------------------
Bret (to Jemaine):

Haha, that's great! It's like they all went to the same therapist school and learned from the same manual. Maybe they have a "Therapists for Dummies" book with all the classic lines. "How does that make you feel?" is definitely a crowd favorite. And you're right, the accent is probably the only thing that changes! But hey, at least they're consistent, right?

-------------------------------------------

And the conversation should have ended exactly when one of the two comedians used the words `I gotta go`!

Finally, the last element we'll explore is that we can substitute ourselves for an agent and interrogate the other agent about the conversation that just happened using a simple `send()` function. For example, let's ask Bret about the last joke Jemaine told him:

In [13]:
jemaine.send(message="What's the last joke we talked about?", recipient=bret)

Jemaine (to Bret):

What's the last joke we talked about?

--------------------------------------------------------------------------------
Bret (to Jemaine):

The last joke we talked about was therapist on a group chat sharing their best "How does that make you feel?" moments, trying to take notes while stifling their laughter, and possibly getting free therapy sessions from our comedy show.

--------------------------------------------------------------------------------
Jemaine (to Bret):

Thanks! Can you believe therapists taking notes at our show? It's like they're trying to analyze our comedy, but really, we're just here to make people laugh. Comedy's not that serious, at least not as serious as therapy. But hey, we're doing our part to spread joy and laughter, one therapist joke at a time. Thanks for the laughs, Bret. I gotta go.

--------------------------------------------------------------------------------


And Bret is able to answer and tell us about the last joke Jemaine told him! We now have agents with states that evolve with the conversation. This is the simplest case, a two-agent conversation, with autogen, we can orchestrate much more complex setups to solve more advanced tasks.

---

## Other examples (homework)

The example we just explored is a simple one that uses jokes, something everybody understands and is familiar with as a support to explain how agents can interact. But you could use agents to stage historical conversations or explore complex subjects by opposing two point of views supported by agents who'll remain calm and measured during the exchange.

Here's an example you could try if you'd like:  

> What if we staged a debate between Satoshi Nakamoto, the creator of Bitcoin and a proponent of hard sound money and Stephanie Kelton one of the main supporters of Modern Monetary Theory, an economic framwork that advocates for easy money to accomplish goals the government deem to be the best for everyone.

Here's a code you coud run if you'd like to try such a scenario:

```python
# Define agent Stephanie
stephanie = ConversableAgent(
    name="Stephanie",
    llm_config=llm_config,
    system_message="Your name is Stephanie Kelton and you are a leading proponent of Modern Monetary Theory. "
    "Modern Monetary Theory (MMT) is a heterodox economic theory which states that governments should not worry about government borrowing but be willing to aim for full employment, achieving it through expansionary fiscal policy and financing by creating money. "
    "You are taking part in a debate about the future of money. "
    "When you're ready to end the conversation, say 'Thank you for having me'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Thank you for having me" in msg["content"],
)

# Define agent Satoshi
satoshi = ConversableAgent(
    name="Satoshi",
    llm_config=llm_config,
    system_message="Your name is Satoshi Nakamoto and you are a the creator of Bitcoin. "
    "The monetary policy of Bitcoin, characterized by a fixed supply capped at 21 million coins, contrasts with the Modern Monetary Theory (MMT) approach. "
    "You are taking part in a debate about the future of money. "
    "When you're ready to end the conversation, say 'Thank you for having me.'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Thank you for having me." in msg["content"],
)

# Initiate the chat
chat_result = stephanie.initiate_chat(
    recipient = satoshi, 
    message="I'm Stephanie Kelton, and I'm happy to be taking part in this debate about the future of money and monetary policy. "
    "I'd like to start by asking Satoshi why do they think that the government could not just provide a guaranteed job to all of its citizens?", 
)
```
